In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

../input/yelp-dataset/yelp_academic_dataset_business.json
../input/yelp-dataset/Dataset_Agreement.pdf
../input/yelp-dataset/yelp_academic_dataset_tip.json
../input/yelp-dataset/yelp_academic_dataset_review.json
../input/yelp-dataset/yelp_academic_dataset_checkin.json
../input/yelp-dataset/yelp_academic_dataset_user.json


In [19]:
import json
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

In [20]:
data = {'useful': [], 'funny': [], 'cool': [],'stars': [], 'text': [],'business':[]}

with open('../input/yelp-dataset/yelp_academic_dataset_review.json') as f:
    for line in tqdm(f):
        review = json.loads(line)
        data['useful'].append(review['useful'])
        data['funny'].append(review['funny'])
        data['cool'].append(review['cool'])
        data['stars'].append(review['stars'])
        data['text'].append(review['text'])
        data['business'].append(review['business_id'])

8021122it [02:00, 66592.98it/s]


In [21]:
df = pd.DataFrame(data)

In [22]:
df['useful'] = df['useful'].astype('int8')
df['funny'] = df['funny'].astype('int8')
df['cool'] = df['cool'].astype('int8')
df['stars'] = df['stars'].astype('int8')
df['text'] = df['text'].astype(str)

In [23]:
#working with just the reviews that have at least 5 votes
df = df[df['useful']+df['funny']+df['cool']>4]

In [24]:
df

,useful,funny,cool,stars,text,business
0,5,0,0,2,"As someone who has worked with many museums, I...",-MhfebM0QIsKt87iDN-FNw
22,24,19,20,4,"Coconut's Fish Cafe is a fantastic, Five Stars...",9Jo1pu0y2zU6ktiwQm6gNA
32,5,3,4,2,This actually used to be one of my favorite ho...,LkMtMHVetws5_7QfRjPtlg
41,5,1,4,3,I stayed here back in April. Friday-Sunday. It...,_ZfjpSEO5ntk-1hbnwCR4g
54,9,5,10,4,Great fresh seafood spot located in the Scotts...,swX3Vj3dQsz1YlCRmwNQEQ
...,...,...,...,...,...,...
8021107,14,0,0,1,I have had two loans go into default with thes...,4EwAkiirgMvFleD3B9HrYg
8021111,9,7,8,2,I was looking for to eating here for quite som...,lvOOJmJ4y0BLi31Muq3a8w
8021114,29,13,4,1,Greedy monks. All I can say\n\nRequires a mini...,bzS33HppGW7eHlFqi1wQGw
8021118,1,3,1,3,Solid American food with a southern comfort fl...,IsoLzudHC50oJLiEWpwV-w


In [25]:
#review is classified as funny if it has at least 50% funny votes
funnies = df[df['funny'] >= df['useful']+df['cool']]

In [26]:
funnies

,useful,funny,cool,stars,text,business
226,2,3,0,5,"Holy ""Made fresh at your table"" Guacamole!!! ...",k-drEjxKmfqllwfY90STfA
440,3,3,0,1,This place leaves a lot to desired. My dad and...,rLTa-PztQLafDf70aKrNOA
545,2,6,0,5,"This town needs an enema!!\n\nWell, here it is...",vpEiM4UuUtbLVibXAHPq3g
647,3,5,1,3,I found this whole concept a little bizarre an...,wpws2fFdsSvyPAsPAfIxaA
685,2,3,0,2,Rich is such a cool guy he could sell just abo...,TFpiquSDJjJET6EZtpcv9g
...,...,...,...,...,...,...
8020858,6,7,1,1,Most horrible customer service ever !!! All th...,3oUqeMtWirI4aOMvQZJwOw
8020896,1,3,2,1,This was the most awkward experience I've ever...,L-BdVyvBq2IN53LIfh-zWQ
8020979,2,5,1,1,Ok I get it. Ross probably isn't the most desi...,wHeG9RYU3_k3wlCgkLGDRA
8021005,4,5,0,1,"Dr Hsiao is very rude and unprofessional, the ...",h8OBD39vzjsQaNicnNDLLw


In [27]:
sorted_funny = funnies.groupby('business').count().sort_values('text',ascending=False)

In [28]:
#There are 46 businesses that have at least 40 'funny' reviews
excluded_funnies = list(sorted_funny[:46].index)

In [29]:
funnies = funnies[~funnies['business'].isin(excluded_funnies)]

In [30]:
funnies

,useful,funny,cool,stars,text,business
226,2,3,0,5,"Holy ""Made fresh at your table"" Guacamole!!! ...",k-drEjxKmfqllwfY90STfA
440,3,3,0,1,This place leaves a lot to desired. My dad and...,rLTa-PztQLafDf70aKrNOA
545,2,6,0,5,"This town needs an enema!!\n\nWell, here it is...",vpEiM4UuUtbLVibXAHPq3g
647,3,5,1,3,I found this whole concept a little bizarre an...,wpws2fFdsSvyPAsPAfIxaA
685,2,3,0,2,Rich is such a cool guy he could sell just abo...,TFpiquSDJjJET6EZtpcv9g
...,...,...,...,...,...,...
8020858,6,7,1,1,Most horrible customer service ever !!! All th...,3oUqeMtWirI4aOMvQZJwOw
8020896,1,3,2,1,This was the most awkward experience I've ever...,L-BdVyvBq2IN53LIfh-zWQ
8020979,2,5,1,1,Ok I get it. Ross probably isn't the most desi...,wHeG9RYU3_k3wlCgkLGDRA
8021005,4,5,0,1,"Dr Hsiao is very rude and unprofessional, the ...",h8OBD39vzjsQaNicnNDLLw


In [31]:
positive = funnies['text']

In [32]:
not_funnies = df[df['funny']==0]

In [33]:
not_funnies

,useful,funny,cool,stars,text,business
0,5,0,0,2,"As someone who has worked with many museums, I...",-MhfebM0QIsKt87iDN-FNw
96,5,0,0,2,The reviews for this place on Facebook are gre...,zRJBKJkEheN30Vl_fdiS7g
165,4,0,2,2,"Hey, I love the Vig Arcadia. I would go there ...",WunR7VclAddvbCnc-97jzg
183,5,0,0,5,I had heard great things about flower child an...,urkODQ1jX0qQzgbqsQjWVQ
197,3,0,3,5,ATTENTION DOWNTown: this place has amazing foo...,tMdgY7Ou4k3bYZZulJCiuQ
...,...,...,...,...,...,...
8020974,8,0,0,2,About Lumiere Chandler\n\nThe community itself...,4CjzDtHbitJsIhuhgsu4Yw
8020981,10,0,0,3,Bought a groupon for botox and filler. Waited ...,IOqYT27UorjB7qTyrEsx3A
8021063,21,0,0,1,MANAGEMENT TRUST EMPLOYEE EMBEZZLED $200K FROM...,12_TcB1midibslKWL7pZKw
8021070,8,0,0,1,Run!!! This place will try to sell you EVERYT...,dqbaOPNOFHFF-4JAlhozQQ


In [34]:
negative = not_funnies[:56561]

In [35]:
negative['label'] = 0

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
negative = negative.drop(['useful','funny','cool','stars','business'],axis = 1)

In [37]:
funnies['label'] = 1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [38]:
positive = funnies.drop(['useful','funny','cool','stars','business'],axis = 1)

In [39]:
positive

,text,label
226,"Holy ""Made fresh at your table"" Guacamole!!! ...",1
440,This place leaves a lot to desired. My dad and...,1
545,"This town needs an enema!!\n\nWell, here it is...",1
647,I found this whole concept a little bizarre an...,1
685,Rich is such a cool guy he could sell just abo...,1
...,...,...
8020858,Most horrible customer service ever !!! All th...,1
8020896,This was the most awkward experience I've ever...,1
8020979,Ok I get it. Ross probably isn't the most desi...,1
8021005,"Dr Hsiao is very rude and unprofessional, the ...",1


In [40]:
negative

,text,label
0,"As someone who has worked with many museums, I...",0
96,The reviews for this place on Facebook are gre...,0
165,"Hey, I love the Vig Arcadia. I would go there ...",0
183,I had heard great things about flower child an...,0
197,ATTENTION DOWNTown: this place has amazing foo...,0
...,...,...
1845514,We love the Overture Center! Been here for cou...,0
1845582,What comes to mind when you think of a fashion...,0
1845613,The place is not the most visually appealing. ...,0
1845646,"I love this place, it was recommended by a fri...",0


In [41]:
all_data = pd.concat([positive,negative])

In [42]:
all_data

,text,label
226,"Holy ""Made fresh at your table"" Guacamole!!! ...",1
440,This place leaves a lot to desired. My dad and...,1
545,"This town needs an enema!!\n\nWell, here it is...",1
647,I found this whole concept a little bizarre an...,1
685,Rich is such a cool guy he could sell just abo...,1
...,...,...
1845514,We love the Overture Center! Been here for cou...,0
1845582,What comes to mind when you think of a fashion...,0
1845613,The place is not the most visually appealing. ...,0
1845646,"I love this place, it was recommended by a fri...",0


In [43]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(all_data['text'],all_data['label'], test_size = 0.1)

In [44]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

In [45]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [46]:
from sklearn.naive_bayes import MultinomialNB

In [47]:
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [48]:
X_test_counts = count_vect.transform(X_test)

In [49]:
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [50]:
predicted = clf.predict(X_test_tfidf)

In [51]:
predicted

array([1, 0, 0, ..., 0, 0, 1])

In [52]:
correct = 0
for pred, actual in zip(predicted,y_test):
    if pred == actual:
        correct += 1

In [53]:
correct/len(predicted)

0.7655794219040043

In [54]:
from sklearn.neighbors import KNeighborsClassifier

In [55]:
clf = KNeighborsClassifier().fit(X_train_tfidf, y_train)

In [56]:
predicted = clf.predict(X_test_tfidf)

In [57]:
correct = 0
for pred, actual in zip(predicted,y_test):
    if pred == actual:
        correct += 1

In [58]:
correct/len(predicted)

0.6304251745779192

In [59]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier().fit(X_train_tfidf, y_train)

In [60]:
predicted = clf.predict(X_test_tfidf)

In [61]:
correct = 0
for pred, actual in zip(predicted,y_test):
    if pred == actual:
        correct += 1
correct/len(predicted)

0.7347299566869973

In [145]:
import matplotlib.pyplot as plt

importances = clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X_train_tfidf.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the impurity-based feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X_train_tfidf.shape[1]), importances[indices],
        color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

Feature ranking:
1. feature 7288 (0.006930)
2. feature 44456 (0.005195)
3. feature 98292 (0.003826)
4. feature 98517 (0.003584)
5. feature 105075 (0.003575)
6. feature 99477 (0.003462)
7. feature 39860 (0.003406)
8. feature 58280 (0.003377)
9. feature 108337 (0.003281)
10. feature 52521 (0.003277)
11. feature 109948 (0.003144)
12. feature 106607 (0.003076)
13. feature 52645 (0.003068)
14. feature 68414 (0.002974)
15. feature 69355 (0.002953)
16. feature 98266 (0.002943)
17. feature 98646 (0.002924)
18. feature 50594 (0.002860)
19. feature 66285 (0.002792)
20. feature 87782 (0.002765)
21. feature 40726 (0.002764)
22. feature 17278 (0.002650)
23. feature 80748 (0.002619)
24. feature 46661 (0.002595)
25. feature 45682 (0.002571)
26. feature 28321 (0.002494)
27. feature 62319 (0.002418)
28. feature 39728 (0.002413)
29. feature 9436 (0.002405)
30. feature 69852 (0.002393)
31. feature 8504 (0.002391)
32. feature 6692 (0.002357)
33. feature 50005 (0.002274)
34. feature 75187 (0.002244)
35. fe

1880. feature 95311 (0.000092)
1881. feature 65097 (0.000092)
1882. feature 85493 (0.000092)
1883. feature 98934 (0.000092)
1884. feature 38393 (0.000092)
1885. feature 19962 (0.000092)
1886. feature 30010 (0.000091)
1887. feature 17300 (0.000091)
1888. feature 99563 (0.000091)
1889. feature 21837 (0.000091)
1890. feature 27882 (0.000091)
1891. feature 43460 (0.000091)
1892. feature 28895 (0.000091)
1893. feature 93018 (0.000091)
1894. feature 50753 (0.000091)
1895. feature 76567 (0.000091)
1896. feature 25973 (0.000091)
1897. feature 18634 (0.000091)
1898. feature 62946 (0.000091)
1899. feature 70219 (0.000091)
1900. feature 97615 (0.000091)
1901. feature 49640 (0.000090)
1902. feature 12592 (0.000090)
1903. feature 77877 (0.000090)
1904. feature 98999 (0.000090)
1905. feature 99668 (0.000090)
1906. feature 93293 (0.000090)
1907. feature 49959 (0.000090)
1908. feature 107043 (0.000090)
1909. feature 10957 (0.000090)
1910. feature 82160 (0.000090)
1911. feature 75778 (0.000090)
1912. f

3379. feature 97963 (0.000037)
3380. feature 47301 (0.000037)
3381. feature 87883 (0.000037)
3382. feature 9605 (0.000037)
3383. feature 1514 (0.000037)
3384. feature 37843 (0.000037)
3385. feature 68145 (0.000037)
3386. feature 97021 (0.000037)
3387. feature 98154 (0.000037)
3388. feature 76979 (0.000037)
3389. feature 82451 (0.000037)
3390. feature 24536 (0.000037)
3391. feature 13990 (0.000036)
3392. feature 11126 (0.000036)
3393. feature 37408 (0.000036)
3394. feature 41455 (0.000036)
3395. feature 100685 (0.000036)
3396. feature 57464 (0.000036)
3397. feature 102758 (0.000036)
3398. feature 88314 (0.000036)
3399. feature 103762 (0.000036)
3400. feature 38593 (0.000036)
3401. feature 50827 (0.000036)
3402. feature 2546 (0.000036)
3403. feature 99931 (0.000036)
3404. feature 87863 (0.000036)
3405. feature 103160 (0.000036)
3406. feature 23590 (0.000036)
3407. feature 8636 (0.000036)
3408. feature 69780 (0.000036)
3409. feature 41540 (0.000036)
3410. feature 95618 (0.000036)
3411. fe

4879. feature 98785 (0.000020)
4880. feature 31351 (0.000020)
4881. feature 97936 (0.000020)
4882. feature 40100 (0.000020)
4883. feature 31031 (0.000020)
4884. feature 75203 (0.000020)
4885. feature 29724 (0.000020)
4886. feature 71707 (0.000020)
4887. feature 23007 (0.000020)
4888. feature 72402 (0.000020)
4889. feature 96174 (0.000020)
4890. feature 107517 (0.000020)
4891. feature 41915 (0.000020)
4892. feature 82449 (0.000020)
4893. feature 11070 (0.000020)
4894. feature 51058 (0.000020)
4895. feature 91742 (0.000020)
4896. feature 88026 (0.000020)
4897. feature 61477 (0.000020)
4898. feature 47508 (0.000020)
4899. feature 35544 (0.000020)
4900. feature 20662 (0.000020)
4901. feature 22475 (0.000020)
4902. feature 9678 (0.000020)
4903. feature 42130 (0.000020)
4904. feature 84427 (0.000020)
4905. feature 100785 (0.000020)
4906. feature 21875 (0.000020)
4907. feature 39522 (0.000020)
4908. feature 16346 (0.000020)
4909. feature 46607 (0.000020)
4910. feature 41393 (0.000020)
4911. f

6378. feature 84426 (0.000012)
6379. feature 2804 (0.000012)
6380. feature 51050 (0.000012)
6381. feature 75907 (0.000012)
6382. feature 97576 (0.000012)
6383. feature 46796 (0.000012)
6384. feature 72839 (0.000012)
6385. feature 15379 (0.000012)
6386. feature 18226 (0.000012)
6387. feature 38076 (0.000012)
6388. feature 82332 (0.000012)
6389. feature 91522 (0.000012)
6390. feature 25747 (0.000012)
6391. feature 4202 (0.000012)
6392. feature 39425 (0.000012)
6393. feature 62913 (0.000012)
6394. feature 18410 (0.000012)
6395. feature 15655 (0.000012)
6396. feature 9652 (0.000012)
6397. feature 28485 (0.000012)
6398. feature 56344 (0.000012)
6399. feature 61622 (0.000012)
6400. feature 94656 (0.000012)
6401. feature 23092 (0.000012)
6402. feature 75471 (0.000012)
6403. feature 45273 (0.000012)
6404. feature 55179 (0.000012)
6405. feature 98038 (0.000012)
6406. feature 1824 (0.000012)
6407. feature 40513 (0.000012)
6408. feature 98559 (0.000012)
6409. feature 79057 (0.000012)
6410. featur

...

109349. feature 60749 (0.000000)
109350. feature 60748 (0.000000)
109351. feature 60747 (0.000000)
109352. feature 60744 (0.000000)
109353. feature 60741 (0.000000)
109354. feature 60740 (0.000000)
109355. feature 60739 (0.000000)
109356. feature 60738 (0.000000)
109357. feature 60735 (0.000000)
109358. feature 60733 (0.000000)
109359. feature 60728 (0.000000)
109360. feature 60727 (0.000000)
109361. feature 60726 (0.000000)
109362. feature 60725 (0.000000)
109363. feature 60936 (0.000000)
109364. feature 60939 (0.000000)
109365. feature 60066 (0.000000)
109366. feature 61188 (0.000000)
109367. feature 61185 (0.000000)
109368. feature 61182 (0.000000)
109369. feature 61181 (0.000000)
109370. feature 61180 (0.000000)
109371. feature 61178 (0.000000)
109372. feature 61176 (0.000000)
109373. feature 61172 (0.000000)
109374. feature 61171 (0.000000)
109375. feature 61167 (0.000000)
109376. feature 61164 (0.000000)
109377. feature 61163 (0.000000)
109378. feature 61161 (0.000000)
109379. fe

110954. feature 59989 (0.000000)
110955. feature 59987 (0.000000)
110956. feature 59985 (0.000000)
110957. feature 59982 (0.000000)
110958. feature 59981 (0.000000)
110959. feature 59979 (0.000000)
110960. feature 59975 (0.000000)
110961. feature 59974 (0.000000)
110962. feature 59972 (0.000000)
110963. feature 59971 (0.000000)
110964. feature 59970 (0.000000)
110965. feature 59969 (0.000000)
110966. feature 59968 (0.000000)
110967. feature 59967 (0.000000)
110968. feature 59966 (0.000000)
110969. feature 59959 (0.000000)
110970. feature 59957 (0.000000)
110971. feature 59953 (0.000000)
110972. feature 59949 (0.000000)
110973. feature 59948 (0.000000)
110974. feature 59942 (0.000000)
110975. feature 59941 (0.000000)
110976. feature 60008 (0.000000)
110977. feature 60010 (0.000000)
110978. feature 59823 (0.000000)
110979. feature 60011 (0.000000)
110980. feature 60060 (0.000000)
110981. feature 60058 (0.000000)
110982. feature 60057 (0.000000)
110983. feature 60054 (0.000000)
110984. fe

NameError: name 'X' is not defined

Error in callback <function flush_figures at 0x7f9bf877d290> (for post_execute):


KeyboardInterrupt: 

In [148]:
inv_map = {v: k for k, v in count_vect.vocabulary_.items()}

In [159]:
inv_map[105075]

'they'

In [63]:
from keras.models import Sequential
from keras.layers import Input, Activation, Dense, Conv2D, MaxPooling2D, Dropout
from keras.optimizers import Adam



In [94]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

train_tok = tokenizer.texts_to_sequences(X_train)
test_tok = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1

In [114]:
X_train[4914100]

'I have always liked Moe\'s.  But unfortunately, this branch is so bad that I don\'t want to come back anymore.  Usually, the staff should greet incoming  customers..."WELCOME TO MOE\'S!!!!"  Not this one!  Staff looks so tired or unhappy being there.  Everything in the line needed to be refilled.  No taco bowl, no steaks, no mushroom...etc! Usually they\'re  fast in making your order. But this one takes a whole month to make one frickin burrito!  Cashier doesn\'t know what he\'s  doing! Their trays are soooooooooo sticky as if they haven\'t been washed in a very long time.  They close 30 mins. before the scheduled hour.  If you come in 30 mins before  the closing time, the staff will tell you they are CLOSING and the bins are already halfway empty!!!!  I hope they change manager and staff........'

In [106]:
train_tok[0]

[3,
 22,
 150,
 663,
 20,
 637,
 15,
 4719,
 11,
 25,
 164,
 13,
 3,
 69,
 116,
 4,
 110,
 59,
 1170,
 443,
 1,
 130,
 171,
 2670,
 361,
 1419,
 4,
 21,
 15,
 39,
 130,
 520,
 25,
 1852,
 43,
 2779,
 153,
 32,
 177,
 8,
 1,
 300,
 308,
 4,
 28,
 3692,
 53,
 895,
 680,
 53,
 2451,
 53,
 1835,
 529,
 443,
 460,
 505,
 8,
 466,
 57,
 113,
 20,
 15,
 39,
 772,
 5,
 349,
 541,
 4,
 107,
 39,
 1050,
 1225,
 339,
 97,
 58,
 874,
 399,
 49,
 5294,
 27,
 2359,
 30,
 31,
 16,
 749,
 73,
 2595,
 8,
 5,
 55,
 200,
 50,
 16,
 431,
 312,
 1113,
 122,
 1,
 1340,
 253,
 31,
 14,
 110,
 8,
 312,
 1113,
 122,
 1,
 1641,
 50,
 1,
 130,
 62,
 245,
 14,
 16,
 27,
 1641,
 2,
 1,
 6564,
 27,
 396,
 3674,
 704,
 3,
 648,
 16,
 513,
 248,
 2,
 130]

In [96]:
len(train_tok[0])

133

In [119]:
from keras.preprocessing.sequence import pad_sequences

train_tok = pad_sequences(train_tok, padding='post', maxlen=200)
test_tok = pad_sequences(test_tok, padding='post', maxlen=200)

In [138]:
from keras.layers import Embedding, GlobalMaxPooling1D,Conv1D
embedding_dim = 50

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=200))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 200, 50)           6147850   
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 196, 128)          32128     
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 128)               0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 10)                1290      
_________________________________________________________________
dropout_10 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_20 (Dense)             (None, 1)               

In [141]:
history = model.fit(train_tok, y_train,
                    epochs=4,
                    verbose=1,
                    validation_split=0.06,
                    batch_size=10)

Train on 95700 samples, validate on 6109 samples
Epoch 1/4
95700/95700 [==============================] - 600s 6ms/step - loss: 0.6088 - accuracy: 0.6672 - val_loss: 0.5469 - val_accuracy: 0.7261
Epoch 2/4
95700/95700 [==============================] - 583s 6ms/step - loss: 0.5501 - accuracy: 0.7334 - val_loss: 0.5331 - val_accuracy: 0.7430
Epoch 3/4
95700/95700 [==============================] - 616s 6ms/step - loss: 0.5216 - accuracy: 0.7532 - val_loss: 0.5299 - val_accuracy: 0.7450
Epoch 4/4
95700/95700 [==============================] - 577s 6ms/step - loss: 0.4947 - accuracy: 0.7713 - val_loss: 0.5345 - val_accuracy: 0.7422


In [142]:
model.evaluate(test_tok,y_test)

11313/11313 [==============================] - 2s 177us/step


[0.5421461842583375, 0.7394148111343384]

In [68]:
model = Sequential()
model.add(Conv2D((10,10),10))
model.add(Activation("relu"))
model.add(MaxPooling2D())
model.add(Dropout(0.5))
model.add(Dense(32))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation("sigmoid"))

model.compile(loss = "binary_crossentropy", optimizer = Adam(lr = 1e-3), metrics = ["accuracy"])
model.fit(np.array(X_train_tfidf),y_train)
model.summary()

ValueError: Input 0 is incompatible with layer conv2d_4: expected ndim=4, found ndim=1